<a href="https://colab.research.google.com/github/dantetarraga/Spark-movie_Recommendation/blob/main/spark_movie-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Librerias a usar

In [46]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer

# Crear una sesión de Spark
spark = SparkSession.builder.appName("MovieRecommender").getOrCreate()



### Sesión APACHE SPARK

In [ ]:
# Crear una sesión de Spark
spark = SparkSession.builder.appName("Sistema_Recomendacion").getOrCreate()

## Conexion y carga de datos

In [ ]:
peliculas_df = spark.read.csv("/movies.csv", header=True, inferSchema=True)
ratings_df = spark.read.csv("/ratings.csv", header=True, inferSchema=True)


## Adaptación Matriz dispersa

In [ ]:
# Join rating y movies con movieId
ratings_df = ratings_df.join(peliculas_df, "movieId")

# Seleccionar las columnas deseadas
ratings_df = ratings_df.select("userId", "movieId", "rating", "title").withColumnRenamed("userId", "user").withColumnRenamed("movieId", "movie")

In [ ]:
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast("float")) # forzamos que rating sea float, generaba error
ratings_df = ratings_df.filter(ratings_df["user"].isNotNull() & ratings_df["movie"].isNotNull() & ratings_df["rating"].isNotNull() & ratings_df["title"].isNotNull()) # descartamos valores nulos o inexistentes

In [ ]:
# Crear StringIndexers para las columnas relevantes
train_data, test_data = ratings_df.randomSplit([0.8, 0.2], seed=42)
user_indexer = StringIndexer(inputCol="user", outputCol="userIndex")
movie_indexer = StringIndexer(inputCol="movie", outputCol="movieIndex")

# Aplicar StringIndexers a los datos de ratings
indexed_data = user_indexer.fit(train_data).transform(train_data)
indexed_data = movie_indexer.fit(indexed_data).transform(indexed_data)

als = ALS(userCol="userIndex", itemCol="movieIndex", ratingCol="rating", nonnegative=True)
model = als.fit(indexed_data)

In [ ]:
user_indexer = StringIndexer(inputCol="user", outputCol="userIndex")
movie_indexer = StringIndexer(inputCol="movie", outputCol="movieIndex")

# Fit StringIndexers and transform the data
indexed_test_data = user_indexer.fit(test_data).transform(test_data)
indexed_test_data = movie_indexer.fit(indexed_test_data).transform(indexed_test_data)

**Paso 7: Evaluar modelo**

In [ ]:
# Generar predicciones en el conjunto de prueba
##predictions = model.transform(indexed_test_data)

In [ ]:
# Evaluar el modelo utilizando una métrica de evaluación (por ejemplo, RMSE)
##evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
##rmse = evaluator.evaluate(predictions)
##print("RMSE: %.2f" % rmse)

In [ ]:
# Generate top 5 recommendations for each user in the test data
recommendations = model.recommendForUserSubset(indexed_test_data, 5)

Mostrar recomendaciones

In [ ]:
# Show the recommendations
recommendations.show(truncate=False)

**Paso 10: Solicitar id de usuario para recomendarle pelicula**

In [ ]:
from pyspark.sql.functions import col

# Solicitar al usuario que ingrese un número
peliculas = input("Ingrese numero de usuario: ") # id_to_retrieve

filtered_data = indexed_data.filter(col("userIndex") == peliculas)
#movie_index = filtered_data.select("movie").collect()[0][0]
movie_index = filtered_data.select("movie").collect()[0][0]
movie_index2 = filtered_data.select("movie").collect()[1][0]
movie_title = (ratings_df.filter(col("movie") == movie_index)).select("title").collect()[0][0]
movie_title2 = (ratings_df.filter(col("movie") == movie_index)).select("title").collect()[1][0]
# Imprimir la recomendación de película para el usuario
print("Para el usuario %s, se recomienda la película: %s" % (peliculas, movie_title))

In [ ]:
train_data, test_data = ratings_df.randomSplit([0.8, 0.2], seed=42)